In [1]:
from keras.models import load_model
import pandas as pd
import numpy as np
import cv2
import os
import re
import sys
from collections import Counter
import seaborn as sns
import pyarrow as pa
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
import multiprocessing
from multiprocessing import Pool
import itertools
from copy import deepcopy
import random

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.decomposition import PCA

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization, GlobalAveragePooling2D, PReLU, GlobalMaxPooling2D
from keras.optimizers import Adam
from keras.utils import to_categorical, Sequence
import keras
import tensorflow as tf
from skimage.transform import AffineTransform, warp

from datetime import datetime
import pickle
import scipy.stats as stats
import os
import sklearn.metrics

Using TensorFlow backend.


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]=""

In [3]:
y = pd.read_csv("data/train.csv")
yEval = pd.read_csv("data/test.csv")
classMap = pd.read_csv("data/class_map.csv")

In [4]:
tables = [pq.read_table('data/train_image_data_{0}.parquet'.format(i)).to_pandas() for i in range(4)]
df = pd.concat(tables)
df = df.set_index("image_id")
del tables

In [5]:
size=(128,128)

In [6]:
class DataLoader(Sequence):
    def __init__(self,X,y,training,batch_size=64,size=(255,255),alpha=3,transform=None,trim="edge"):
        self.training = training
        self.batch_size=batch_size
        self.X=X
        self.y=y
        self.size=size
        self.alpha=alpha
        self.transform=transform
        self.trim=trim

    def __len__(self):
        return int(np.ceil(self.X.shape[0] / self.batch_size))


    def __getitem__(self, idx):
        
        _imgs=self.X[idx * self.batch_size:(idx + 1) * self.batch_size,:,:]
        
        imgs=[]
        for img in _imgs:
            imgs.append(transformImg(img,size=self.size,training=self.training,trim=self.trim))
        
        if self.transform is not None:
            _imgs=[]
            for img in imgs:
                if np.random.random()<0.5:
                    res = self.transform(image=img)
                    img = res['image'].astype(np.float32)
                else:
                    pass
                _imgs.append(img)
            imgs=_imgs 
        
        imgs=np.asarray(imgs)
        
        
        ret_y=[]
        for label in labels:
            ret_y.append(to_categorical(self.y[idx * self.batch_size:(idx + 1) * self.batch_size][label],num_classes=len(set(y[label]))))
    
        #mix up
        if self.training :
            r = np.random.permutation(imgs.shape[0])
            imgs2=deepcopy(imgs)[r]
            grapheme=ret_y[0]
            vowel=ret_y[1]
            consonant=ret_y[2]
            grapheme2=deepcopy(grapheme)[r]
            vowel2=deepcopy(vowel)[r]
            consonant2=deepcopy(consonant)[r]
            ratio=np.random.beta(self.alpha,self.alpha,imgs.shape[0])
            ratio[ratio>1]=1
            ratio[ratio<0]=0
            imgs=np.tile(ratio,(3,*size,1)).T*imgs+np.tile((1-ratio),(3,*size,1)).T*imgs2
            grapheme=np.tile(ratio,(168,1)).T*grapheme+np.tile((1-ratio),(168,1)).T*grapheme2
            vowel=np.tile(ratio,(11,1)).T*vowel+np.tile((1-ratio),(11,1)).T*vowel2
            consonant=np.tile(ratio,(7,1)).T*consonant+np.tile((1-ratio),(7,1)).T*consonant2
            grapheme=grapheme.astype(np.float32)
            vowel=vowel.astype(np.float32)
            consonant=consonant.astype(np.float32)
            ret_y=[grapheme,vowel,consonant]
   
        if self.training:
            imgs = [cutout(img) for img in imgs]
            pass
            
        imgs = np.asarray(imgs).astype(np.float32)/255.0
            

        return imgs, ret_y

def transformImg(img,size=(255,255),training=True,clahe=False,mergin=5,trim="edge"):
    ret2, img = cv2.threshold(img, 0, 255, cv2.THRESH_OTSU)
    img = 255-img
    if training:
        img=affine_image(img)
    if trim=="edge":
        img = img[upper(img,mergin):lower(img,mergin),lefter(img,mergin):righter(img,mergin)]
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    img = cv2.resize(img, (size[0],size[1]))
    kernel = np.ones((3,3),np.float32)/9
    img = cv2.filter2D(img,-1,kernel)
    return img

def cutout(image_origin):
    # 最後に使うfill()は元の画像を書き換えるので、コピーしておく
    image = np.copy(image_origin)
    mask_value = image.mean()

    h, w, c = image.shape
    # マスクをかける場所のtop, leftをランダムに決める
    # はみ出すことを許すので、0以上ではなく負の値もとる(最大mask_size // 2はみ出す)
    mask_size=h//2
    top = np.random.randint(0 - mask_size // 2, h - mask_size)
    left = np.random.randint(0 - mask_size // 2, w - mask_size)
    bottom = top + mask_size
    right = left + mask_size

    # はみ出した場合の処理
    if top < 0:
        top = 0
    if left < 0:
        left = 0

    # マスク部分の画素値を平均値で埋める
    image[top:bottom, left:right, :].fill(mask_value)
    return image

In [7]:
labels = ["grapheme_root","vowel_diacritic","consonant_diacritic"]

In [8]:
X_train, X_test, y_train, y_test = train_test_split(df.values.reshape(-1,137,236), y, train_size=0.9, random_state=8888)
gen = DataLoader(X_test, y_test, training=False, batch_size=512, size=size,trim=None)
X_train, X_test, y_train, y_test = train_test_split(df.values.reshape(-1,137,236), y, train_size=0.9, random_state=8000)
gen2 = DataLoader(X_test, y_test, training=False, batch_size=512, size=size,trim=None)

In [9]:
model=load_model("multiRes152V2-justresize.h5",compile=False)
model.compile(optimizer=Adam(), metrics=["acc"], loss="categorical_crossentropy", loss_weights=[2.,1.,1.])
print(model.evaluate(gen,verbose=1))
print(model.evaluate(gen2,verbose=1))

40/40 [==============================] - 382s 10s/step
[0.05067349225282669, 0.083149753510952, 0.03222891315817833, 0.029337456449866295, 0.9748058319091797, 0.9920334815979004, 0.9917347431182861]
40/40 [==============================] - 374s 9s/step
[0.42937761545181274, 0.14010223746299744, 0.047822363674640656, 0.04180247336626053, 0.9602170586585999, 0.9891455769538879, 0.9890459775924683]


In [10]:
model.save("multiRes152V2-justresize.h5",include_optimizer=False)

In [ ]:
import functools
def get_swish(**kwargs):
    backend, layers, models, keras_utils = get_submodules_from_kwargs(kwargs)
    def swish(x):
        """Swish activation function: x * sigmoid(x).
        Reference: [Searching for Activation Functions](https://arxiv.org/abs/1710.05941)
        """

        if backend.backend() == 'tensorflow':
            try:
                # The native TF implementation has a more
                # memory-efficient gradient implementation
                return backend.tf.nn.swish(x)
            except AttributeError:
                pass

        return x * backend.sigmoid(x)
    return  swish


def get_dropout(**kwargs):
    backend, layers, models, keras_utils = get_submodules_from_kwargs(kwargs)
    class FixedDropout(layers.Dropout):
        def _get_noise_shape(self, inputs):
            if self.noise_shape is None:
                return self.noise_shape

            symbolic_shape = backend.shape(inputs)
            noise_shape = [symbolic_shape[axis] if shape is None else shape
                           for axis, shape in enumerate(self.noise_shape)]
            return tuple(noise_shape)

    return FixedDropout
_KERAS_BACKEND = None
_KERAS_LAYERS = None
_KERAS_MODELS = None
_KERAS_UTILS = None


def get_submodules_from_kwargs(kwargs):
    backend = kwargs.get('backend', _KERAS_BACKEND)
    layers = kwargs.get('layers', _KERAS_LAYERS)
    models = kwargs.get('models', _KERAS_MODELS)
    utils = kwargs.get('utils', _KERAS_UTILS)
    for key in kwargs.keys():
        if key not in ['backend', 'layers', 'models', 'utils']:
            raise TypeError('Invalid keyword argument: %s', key)
    return backend, layers, models, utils


def inject_keras_modules(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        kwargs['backend'] = keras.backend
        kwargs['layers'] = keras.layers
        kwargs['models'] = keras.models
        kwargs['utils'] = keras.utils
        return func(*args, **kwargs)

    return wrapper

backend = None
layers = None
models = None
keras_utils = None
def generalized_mean_pool_2d(X):
    import tensorflow as tf
    gm_exp = tf.Variable(3.0, dtype = tf.float32)
    pool = (tf.reduce_mean(tf.abs(X**(gm_exp)), 
                        axis = [1, 2], 
                        keepdims = False) + 1.e-7)**(1./gm_exp)
    return pool


gm_exp = tf.Variable(3.0, dtype = tf.float32)

In [ ]:
custom_objects= {'swish': inject_keras_modules(get_swish)(),'FixedDropout': inject_keras_modules(get_dropout)(),"tf":tf,"gm_exp":gm_exp}